# Arcjet 脱敏

[Arcjet](https://arcjet.com) 脱敏集成允许您在将提示发送给LLM之前，从中脱敏敏感的用户信息。

Arcjet 脱敏完全在您自己的机器上运行，从不将数据发送到其他任何地方，确保了最佳的隐私和性能。

Arcjet 脱敏对象本身不是一个LLM，而是封装了一个LLM。它会脱敏输入的文本，然后在返回之前对封装的LLM的输出进行反脱敏。



## 概述
### 集成详情

| 类 | 包 | 本地 | 可序列化 | Python支持 | 包下载量 | 最新包 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| Arcjet | @langchain/community | ❌ | ✅ | ❌ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/community?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

### 安装

安装Arcjet脱敏库：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @arcjet/redact
</Npm2Yarn>

```

并安装LangChain Community：


```{=mdx}
<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>

现在，您就可以开始使用Arcjet脱敏保护您的LLM调用了！

```

## 用法

In [1]:
import {
  ArcjetRedact,
  ArcjetSensitiveInfoType,
} from "@langchain/community/llms/arcjet";
import { OpenAI } from "@langchain/openai";

// Create an instance of another LLM for Arcjet to wrap
const openai = new OpenAI({
  modelName: "gpt-3.5-turbo-instruct",
  openAIApiKey: process.env.OPENAI_API_KEY,
});

const arcjetRedactOptions = {
  // Specify a LLM that Arcjet Redact will call once it has redacted the input.
  llm: openai,

  // Specify the list of entities that should be redacted.
  // If this isn't specified then all entities will be redacted.
  entities: ["email", "phone-number", "ip-address", "credit-card"] as ArcjetSensitiveInfoType[],

  // You can provide a custom detect function to detect entities that we don't support yet.
  // It takes a list of tokens and you return a list of identified types or undefined.
  // The undefined types that you return should be added to the entities list if used.
  detect: (tokens: string[]) => {
    return tokens.map((t) => t === "some-sensitive-info" ? "custom-entity" : undefined)
  },

  // The number of tokens to provide to the custom detect function. This defaults to 1.
  // It can be used to provide additional context when detecting custom entity types.
  contextWindowSize: 1,

  // This allows you to provide custom replacements when redacting. Please ensure
  // that the replacements are unique so that unredaction works as expected.
  replace: (identifiedType: string) => {
    return identifiedType === "email" ? "redacted@example.com" : undefined;
  },

};

const arcjetRedact = new ArcjetRedact(arcjetRedactOptions);
const response = await arcjetRedact.invoke(
  "My email address is test@example.com, here is some-sensitive-info"
);